In [111]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import numpy as np # linear algebra
import pandas # data processing, CSV file I/O (e.g. pd.read_csv)
from pprint import pprint
import time
import io
import os
import re
from pprint import pprint
g = 0

firstyear = 2021
finalyear = 2021
league = 'N'

just_this_team = 'WAS' #['ANA', 'BAL']
just_this_team_order = 14


pandas.set_option('display.max_colwidth', None)

eventdir = '../input/baseball-events-from-retrosheetorg/'

processed_data_dir = '../input/baseball-events-from-retrosheetorg/processed/processed/'

save_fielders = True

print('initializing dataframes...')
event_id_columns = ['eventID']
gamecolumns = ['gameID']
info_columns = gamecolumns + ['visteam', 'hometeam', 'site', 'date', 'number', 'starttime', 'daynight', 'usedh', 'innings', 'tiebreaker', 'umphome', 'ump1b', 'ump2b', 'ump3b', 'umplf', 'umprf', 'inputtime', 'howscored', 'pitches', 'oscorer', 'temp', 'winddir', 'windspeed', 'fieldcond', 'precip', 'sky', 'timeofgame', 'attendance', 'wp', 'lp', 'save', 'comments']
player_columns = ['playerID', 'visitor_or_home', 'batting_order', 'fielding_position']

event_columns = event_id_columns + gamecolumns
event_columns += ['event_in_game', 'inning', 'visitor_or_home', 'batterID', 'batter_hand']
event_columns += ['pitch_count', 'pitch_sequence', 'theplay']
event_columns += ['pitcherID', 'pitcher_hand', 'badj', 'padj', 'ladj']
if (save_fielders):
    event_columns += ['field_C_playerID', 'field_1B_playerID', 'field_2B_playerID', 'field_3B_playerID', 'field_SS_playerID', 'field_LF_playerID', 'field_CF_playerID', 'field_RF_playerID', 'bat_DH_playerID']
event_columns += ['pinch_runner_playerID', 'pinch_runner_replaced_playerID']#, 'badj', 'padj']
event_columns += ['comments']

def ordinaltg(n):
    return str(n) + {1: 'st', 2: 'nd', 3: 'rd'}.get(4 if 10 <= n % 100 < 20 else n % 10, "th")

g = 0
print('Packages loaded!')

initializing dataframes...
Packages loaded!


In [115]:
def add_roster_entry_if_needed(rosters_df, theplayerID, thenameLast, thenameFirst, thebatting_hand, thethrowing_hand, theteam, theposition, thetheyear):
    if not(theplayerID in rosters_df):
        rosters_df.loc[rosters_df.index.max()+1] = [theplayerID, thenameLast, thenameFirst, thebatting_hand, thethrowing_hand, theteam, theposition, thetheyear]
    else:
        print('{0:} ({1:} {2:}) already in rosters file...'.format(theplayerID, thenameFirst, thenameLast))
        pprint(rosters_df.loc[playerID])
        print('----------------------------')
    return


def add_special_case_roster_entries(rosters_df):
### SEA
    add_roster_entry_if_needed(rosters_df, 'arozr001', 'Arozarena', 'Randy', 'R', 'R', 'TBA', 'OF', 2021)
    add_roster_entry_if_needed(rosters_df, 'astuw001', 'Astudillo', 'Willians', 'R', 'R', 'MIN', 'C', 2021)
    add_roster_entry_if_needed(rosters_df, 'beltb001', 'Belt', 'Brandon', 'L', 'L', 'SFN', '1B', 2021)
    add_roster_entry_if_needed(rosters_df, 'cabrm001', 'Cabrera', 'Miguel', 'R', 'R', 'DET', '1B', 2021)
    add_roster_entry_if_needed(rosters_df, 'campe001', 'Campbell', 'Eric', 'R', 'R', 'SEA', '3B', 2021)
    add_roster_entry_if_needed(rosters_df, 'canhm001', 'Canha', 'Mark', 'R', 'R', 'OAK', 'OF', 2021)
    
    
    add_roster_entry_if_needed(rosters_df, 'castw003', 'Castro', 'Willi', 'B', 'R', 'DET', 'SS', 2021)
    add_roster_entry_if_needed(rosters_df, 'chany001', 'Chang', 'Yu', 'R', 'R', 'CLE', 'SS', 2021)
    add_roster_entry_if_needed(rosters_df, 'dawsr001', 'Dawson', 'Ronnie', 'L', 'L', 'HOU', 'SS', 2021)
    add_roster_entry_if_needed(rosters_df, 'diaze005', 'Diaz', 'Elias', 'R', 'R', 'COL', 'C', 2021)
    add_roster_entry_if_needed(rosters_df, 'dozih001', 'Dozier', 'Hunter', 'R', 'R', 'KCA', 'OF', 2021)
    add_roster_entry_if_needed(rosters_df, 'eatoa002', 'Eaton', 'Adam', 'L', 'L', 'CHA', 'OF', 2021)
    add_roster_entry_if_needed(rosters_df, 'garca006', 'Garcia', 'Aramis', 'R', 'R', 'OAK', 'C', 2021)
    add_roster_entry_if_needed(rosters_df, 'godoj001', 'Godoy', 'Jose', 'L', 'R', 'SEA', 'C', 2021)
    add_roster_entry_if_needed(rosters_df, 'gordn001', 'Gordon', 'Nick', 'L', 'R', 'MIN', 'SS', 2021)
    add_roster_entry_if_needed(rosters_df, 'hickj001', 'Hicks', 'John', 'R', 'R', 'TEX', 'C', 2021)
    add_roster_entry_if_needed(rosters_df, 'judga001', 'Judge', 'Aaron', 'R', 'R', 'NYA', 'OF', 2021)
    add_roster_entry_if_needed(rosters_df, 'kierk001', 'Kiermaier', 'Kevin', 'L', 'R', 'TBA', 'OF', 2021)
    add_roster_entry_if_needed(rosters_df, 'lopej005', 'Lopez', 'Jack', 'R', 'R', 'BOS', 'SS', 2021)
    add_roster_entry_if_needed(rosters_df, 'maldm001', 'Maldonado', 'Martin', 'R', 'R', 'HOU', 'C', 2021)
    add_roster_entry_if_needed(rosters_df, 'marsb002', 'Marsh', 'Brandon', 'L', 'R', 'ANA', 'OF', 2021)
    add_roster_entry_if_needed(rosters_df, 'merrw001', 'Merrifield', 'Whit', 'R', 'R', 'KCA', 'OF', 2021)
    add_roster_entry_if_needed(rosters_df, 'naylj001', 'Naylor', 'Josh', 'L', 'L', 'CLE', 'OF', 2021)
    add_roster_entry_if_needed(rosters_df, 'riose001', 'Rios', 'Edwin', 'L', 'R', 'LAN', '3B', 2021)
    add_roster_entry_if_needed(rosters_df, 'solad001', 'Solano', 'Donovan', 'R', 'R', 'SFN', '2B', 2021)
    add_roster_entry_if_needed(rosters_df, 'vauga001', 'Vaughn', 'Andrew', 'R', 'R', 'CHA', 'OF', 2021)
    add_roster_entry_if_needed(rosters_df, 'barlj001', 'Barlow', 'Joe', 'R', 'R', 'TEX', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'bassc001', 'Bassitt', 'Chris', 'R', 'R', 'OAK', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'bielb001', 'Bielak', 'Brandon', 'L', 'R', 'HOU', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'duggr001', 'Dugger', 'Robert', 'R', 'R', 'SEA', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'dolir001', 'Dolis', 'Rafael', 'R', 'R', 'TOR', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'evand003', 'Evans', 'Demarcus', 'R', 'R', 'TEX', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'fairp001', 'Fairbanks', 'Pete', 'R', 'R', 'TBA', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'guerj003', 'Guerra', 'Junior', 'R', 'R', 'ANA', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'hernc006', 'Hernandez', 'Carlos', 'R', 'R', 'KCA', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'kopem001', 'Kopech', 'Michael', 'R', 'R', 'CHA', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'margn001', 'Margevicius', 'Nick', 'L', 'L', 'SEA', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'may-d003', 'May', 'Dustin', 'R', 'R', 'LAN', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'ramiy002', 'Ramirez', 'Yohan', 'R', 'R', 'SEA', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'roblh001', 'Robles', 'Hansel', 'R', 'R', 'MIN', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'santd003', 'Santana', 'Dennis', 'R', 'R', 'TEX', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'sawah001', 'Sawamura', 'Hirokazu', 'R', 'R', 'BOS', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'stanr002', 'Stanek', 'Ryne', 'R', 'R', 'HOU', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'stept002', 'Stephan', 'Trevor', 'R', 'R', 'CLE', 'P', 2021)
### TEX
    add_roster_entry_if_needed(rosters_df, 'grist001', 'Grisham', 'Trent', 'L', 'L', 'SDN', 'OF', 2021)
    add_roster_entry_if_needed(rosters_df, 'holab001', 'Holaday', 'Bryan', 'R', 'R', 'ARI', 'C', 2021)
    add_roster_entry_if_needed(rosters_df, 'profj001', 'Profar', 'Jurickson', 'B', 'R', 'SDN', '2B', 2021)
    add_roster_entry_if_needed(rosters_df, 'santa003', 'Santander', 'Anthony', 'B', 'R', 'BAL', 'OF', 2021)
    add_roster_entry_if_needed(rosters_df, 'borur001', 'Borucki', 'Ryan', 'L', 'L', 'TOR', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'brica001', 'Brice', 'Austin', 'R', 'R', 'BOS', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'coleg001', 'Cole', 'Gerrit', 'R', 'R', 'NYA', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'cottj001', 'Cotton', 'Jharel', 'R', 'R', 'TEX', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'dobnr001', 'Dobnak', 'Randy', 'R', 'R', 'MIN', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'garcb001', 'Garcia', 'Bryan', 'R', 'R', 'DET', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'jimej003', 'Jimenez', 'Joe', 'R', 'R', 'DET', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'leond003', 'Leone', 'Dominic', 'R', 'R', 'SFN', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'gonzr003', 'Gonzalez', 'Roman', 'R', 'R', 'CHA', 'IF', 2021)
    add_roster_entry_if_needed(rosters_df, 'shorz001', 'Short', 'Zack', 'R', 'R', 'DET', 'SS', 2021)
    add_roster_entry_if_needed(rosters_df, 'bowdb001', 'Bowden', 'Ben', 'L', 'L', 'COL', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'lawrj002', 'Lawrence', 'Justin', 'R', 'R', 'COL', 'P', 2021)
    add_roster_entry_if_needed(rosters_df, 'rodrc002', 'Rodriguez', 'Chris', 'R', 'R', 'ANA', 'P', 2021)

    
    return rosters_df


def fix_special_case_hands(events_df):
    #### 2021
    ### ANA
    events_df.loc[events_df['batterID'] == 'adelj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'arozr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'astuw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'beltb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bemba001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bundd001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'buted001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cabrm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'canhm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'canng001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'castw003', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'chany001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cishs001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'diaze005', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dozih001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'eatoa002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fletd002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fowld001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'garca006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gossp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grist001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'iglej001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'igler001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jay-j001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'judga001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kierk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'lagaj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'maldm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mayem001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mayfj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'merrw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'naylj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ohtas001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'profj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'pujoa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'quijj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'renda001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'rengl001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'rizza001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'rojaj002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'santa003', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'sches001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'selms001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'solad001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stasm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'suzuk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'thaim001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'troum001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'uptoj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'walsj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wardt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'watst001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wilsj005', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wongk002', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'astuw001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bassc001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bolar001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'coleg001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'cottj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dobnr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'duggr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'evand003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'garcb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'guerj003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'jimej003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'kopem001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'leond003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ramiy002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'roblh001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'santd003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'stanr002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'godoj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'marsb002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'vauga001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'barlj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bowdb001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'crisc002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'lawrj002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'rodrc002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'sawah001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'stept002', 'pitcher_hand'] = 'R'
    ### BAL
    events_df.loc[events_df['batterID'] == 'andeb006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ciufn001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'contw002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'franm004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'freef001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'galvf001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'gutik001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'haysa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hoyij001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'jonej007', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'leybd001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'manct001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'martr006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'matej003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mcker001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mounr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mullc002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'mullc002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'nevit001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ruizr002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'sevep001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'siscc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'stewd002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'uriar001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'valap001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilks001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wynna001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'barnj002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'beasj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bielb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bleir001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'brica001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'charj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dolir001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ellic001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'fairp001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'garcd003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'garzr001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'krisb001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'margn001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'pearn001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'tapid001', 'pitcher_hand'] = 'R'    
    events_df.loc[events_df['batterID'] == 'burgj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gordn001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'shorz001', 'batter_hand'] = 'R'    
    events_df.loc[events_df['pitcherID'] == 'barrm004', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clays001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'hernc006', 'pitcher_hand'] = 'R'    
    ### BOS
    events_df.loc[events_df['batterID'] == 'arauj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'arroc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bogax001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'branr002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chavm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coopg002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cordf003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dalbb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dever001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gonzm002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'herne001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'johnd006', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'martj006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mottt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'plawk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'renfh001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'santd001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'schwk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'shawt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'vazqc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'verda001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wongc001', 'batter_hand'] = 'R'    
    events_df.loc[events_df['pitcherID'] == 'borur001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'castm002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'deloe001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'duraj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'lopej005', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bazae001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'garzj001', 'pitcher_hand'] = 'R'
    ### CHA
    events_df.loc[events_df['batterID'] == 'abrej003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'andet001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'barnt001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'carpm002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'collz001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'duffm002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'engea001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garcl004', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'goldp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gonzl005', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'goodb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'grany001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'hamib001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'hernc005', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'jimee001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lambj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'madrn001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mendd002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mercy001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'moncy001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'robel002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'sheeg001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'sosae001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willn001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zavas001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gonzr003', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gonzr003', 'pitcher_hand'] = 'R'
    ### CLE
    events_df.loc[events_df['batterID'] == 'bauej001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bradb002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bryak001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cleme002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gameb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gimea001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hedga001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lavar001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'luplj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'merco003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'millo002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'pedej001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'perer003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ramih004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ramij003', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'ramow001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'reyef001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'river003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'rosaa003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'rosae001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'stram002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zimmb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'burnc002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'castl003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'popps001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'romas001', 'pitcher_hand'] = 'R'
    ### DET
    events_df.loc[events_df['batterID'] == 'badda001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'camed001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'candj002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'casth001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'garnd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gonze004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'goodn002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'greig001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grosr001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'haase001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hickj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hilld002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jonej006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mazan001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'nuner001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'parei001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'reyev001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'robsj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'rogej004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'schoj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'arrij001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'burrb002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bednd001', 'pitcher_hand'] = 'R'
### HOU
    events_df.loc[events_df['batterID'] == 'altuj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'alvay001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'branm003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'brega001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'castj006', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'corrc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'diaza003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garcr005', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'gurry001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jonet005', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kozmp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mccoc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'meyej002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'schwf001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'sirij001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stubg001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'toroa001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'tuckk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'lamed001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'ramin002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dawsr001', 'batter_hand'] = 'L'
### KCA
    events_df.loc[events_df['batterID'] == 'albeh001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'benia002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dysoj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gallc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'isbek001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'lopen001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'mcbrr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mondr003', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'ohear001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'olive001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'peres002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'rivee001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'rives002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'santc002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'solej001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stauj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'taylm002', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gutiv001', 'pitcher_hand'] = 'R'
### MIN
    events_df.loc[events_df['batterID'] == 'arral001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'buxtb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cavej001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'celeg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cruzn002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'donaj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garlk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garvm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jeffr002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'keplm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'kiria001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'larnt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'polaj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'refsr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'riddj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'rookb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'rortb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'sanom001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'simma001', 'batter_hand'] = 'R'
### NYA
    events_df.loc[events_df['batterID'] == 'alleg002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'ambut001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'andum001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brucj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'davij007', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'flore001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fordm002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'frazc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gallj002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gardb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gittc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gomey001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hicka001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'higak001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lamar001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lemad001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'locat001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'odorr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'parkh001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'sancg002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stanm004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'taucm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'torrg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'urshg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'velaa001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'voitl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wadet002', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'andei001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'oswac001', 'pitcher_hand'] = 'R'
### OAK
    events_df.loc[events_df['batterID'] == 'allea001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'andre001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'barrl001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bellc002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bolts001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'brows003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'campe001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chapm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davik003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'escoe001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'harrj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kempt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'laurr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lowrj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'machv001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'marts002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'morem001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'murps001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'olsom001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'pindc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'piscs001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'riose001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'tom-k001', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'may-d003', 'pitcher_hand'] = 'R'
### SEA - players recorded in roster file as R or L who actually hit switch
    events_df.loc[events_df['batterID'] == 'bolts001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'galvf001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'hernc005', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'toroa001', 'batter_hand'] = 'B'
### TBA
    events_df.loc[events_df['batterID'] == 'adamw002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brosm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brujv001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'choij001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'diazy001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'franw002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'loweb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'margm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'meada001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'mejif001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'padlk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'philb002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'quinr003', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'smitk002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tsuty001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wallt003', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'wendj002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zunim001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fortn001', 'batter_hand'] = 'R'
### TOR
    events_df.loc[events_df['batterID'] == 'adamr004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bichb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'biggc002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dickc002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'espis001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gricr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'guerv002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gurrl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hernt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jansd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kirka001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mcgur002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'palaj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'panij002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'semim001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'smitk003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'sprig001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tellr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'valeb001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'martl005', 'batter_hand'] = 'R'
#### 2021 NL
### ARI
    events_df.loc[events_df['batterID'] == 'aguim001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ahmen001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'andei001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bleir001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'buchr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bumgm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'burna003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cabra002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'calhk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'casth002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'castm002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clart003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clipt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crics001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'degeb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deloe001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'devec001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'duplj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ellid002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fairs001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'farij001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fishd001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'frans001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gallz001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gibsk002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gilbt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ginkk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'givem001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hagej001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'heatn002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'holab001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kellc002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kellm002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lamed001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lopey001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mantj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'martc006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'martc009', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'martk001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'mathw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mccaj003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'mejih001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'peacm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'perad001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'perdg001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'popps001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ramin002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ramoh001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'reddj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'rojaj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'romas001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'sittb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'smitc006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'smitp002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'smitr005', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'sorij001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'swara001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vanmj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'vargi001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'varsd001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'vogts001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'walkc002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'weavl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wendj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'widet001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'youna002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'youna003', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'clart003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'gibsk002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'givem001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'martc006', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bukaj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'bukaj001', 'pitcher_hand'] = 'R'
### ATL
    events_df.loc[events_df['batterID'] == 'acunr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'adrie001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'albio001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'almoa001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'arcio002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'biddj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'burnc002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'camaj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'chavj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'darnt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davit005', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'daytg001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'duvaa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'friem001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'grees005', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hereg002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'incie001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'jacka002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jackl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jonen001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'martc007', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mathj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'matzt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'minta001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'mortc002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mullk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'newcs001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ozunm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'pachc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'rilea001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'rodrr008', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'sandp001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'sandp001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'sante002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'smitw002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'smyld001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'stris002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'swand001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tomlj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'toust001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'webbj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsb003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ynoah001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'duffm002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'duffm002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gutiv001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'klobg001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'lee-d005', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'klobg001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'lee-d005', 'pitcher_hand'] = 'L'
### CHN
    events_df.loc[events_df['batterID'] == 'abboc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'adamj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'alcas002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'alzoa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'arrij001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'baezj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'barnj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'boted002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brotr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'castl003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chafa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chirr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'contw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'daviz001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deicg001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'effrs001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fargj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'felim001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'happi001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'hendk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hermm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'heuec001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'heywj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hoern001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jewej001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kimbc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lobaj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'mapld001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'marij002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'martn003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'megit001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'meisr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'milla003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'morga001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'nanct001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'orter001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'rivaa001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'rodrm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'romia001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'romia002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ruckm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ryank002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'sampa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'sogae001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'steej002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'stewk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'strop001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'teper001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'thomk002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'thomt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wickr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wiecb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'winkd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wisdp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'woltt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'workb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'felim001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bednd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'clays001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dovac001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'elles001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'higgp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'owenh003', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'dovac001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'elles001', 'pitcher_hand'] = 'R'
### CIN
    events_df.loc[events_df['batterID'] == 'akiys001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'alanr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'antot001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'aquia001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'blana002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bracb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'burrb002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'castn001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cessl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davit004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'delej003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'deshd002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dools001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'eickj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'farmk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'freem002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'friet001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fulmc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garce001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garcj007', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garra001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gouda001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grays001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'heins001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hembh001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hendr003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hoffj003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'indij001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lopea005', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'lorem002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mahlt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'milew001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'mousm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'naqut001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'osicj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'paytm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'perec004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'perec004', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'rogej003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'sanmr001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'santt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'schrm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'senzn001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'simsl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stept001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'suare001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vottj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'warra002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsj004', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'winkj002', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'eickj001', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'rogej003', 'pitcher_hand'] = 'L'
### COL
    events_df.loc[events_df['batterID'] == 'adamm002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'almoy001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'almoy001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'bardd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'blacc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bowdb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'castr005', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chacj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cronc002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dazay001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'estec001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fernj007', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'freek001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fuenj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gilbl001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gomba001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gonzc002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grayj003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hampg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hills005', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'joe-c001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kinlt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lambp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lawrj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'marqg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mcmar001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'nuned001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'owinc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'rodgb002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'santa004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'senza001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'shefj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stepr002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stort001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tapir001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'treja001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'welkc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'castk002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vilar001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'castk002', 'pitcher_hand'] = 'R'
### LAN
    events_df.loc[events_df['batterID'] == 'alexs001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'barna001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bauet001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'beatm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bettm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bickp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bruij001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'buehw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cleag001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gonst001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gonzv001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gratb002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'grayj004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hillr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'jacka003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jansk001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'kellj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kersc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'knebc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lux-g001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'may-d003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mckib001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'mckiz001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'muncm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'nelsj004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'neuss001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'nuned002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'peted004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'phile001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'polla001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'pricd001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ralel001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'reksz001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ruizk001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'santd003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'schem001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'seagc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'sherj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'smitw003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'souzs001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'taylc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'treib001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'turnj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'turnt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ucete001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'uriaj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'vesia001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'whitm004', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'avill001', 'pitcher_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'burna003', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'hillr001', 'pitcher_hand'] = 'L'
### MIA
    events_df.loc[events_df['batterID'] == 'aguij001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'alcas001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'alfaj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'alvae001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'bassa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bella001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'benda001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bertj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brinl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cabre003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'campp001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'castd003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'chisj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cimba001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'curtj003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'delab001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'detwr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'devej001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'diazi001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'diazl001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'flord002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'garcy001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garrb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'garrb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'guens001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'guilp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'henrp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'herne002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hessd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hollj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'leons001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'lopep002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'luzaj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'madel001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'marrd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'millb003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'mitcb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'moris001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'neidn001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'okers001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'pop-z001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'potec001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'pruia001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'roget003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'rojam002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'sancj003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'sierm002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'thomz001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wallc001', 'batter_hand'] = 'R'
### MIL
    events_df.loc[events_df['batterID'] == 'andeb004', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'ashba003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'betta001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'boxbb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bradj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cainl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cousj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'felim002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'feyej001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garca003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'godlz001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gustj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hadej001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hiurk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'housa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lauee001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lindj004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lopet001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'maill001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'milnh001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'narvo001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'norrd002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'nottj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'peraf001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'perda001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'petej002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'pinam001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'rasmd002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'reyep001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'richt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'robed004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'sancm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'strih001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'suteb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'taylt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'urial001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'voged001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'weigp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willd004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wongk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodb005', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'yarde001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yelic001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'zimmj003', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'axfoj001', 'pitcher_hand'] = 'R'
### NYN
    events_df.loc[events_df['batterID'] == 'almoa002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'alonp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'banda001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'blant002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'bosta001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carrc003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'confm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'davij006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'degrj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'diaze006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'diazy002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'drurb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'famij001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gselr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'guill001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'handb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hartg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'huntt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lee-k001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'lindf001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'loupa001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'luccj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'lugos001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'may-t001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'maybc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mazep001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'mccaj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mcnej002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'megit002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'nidot001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'nimmb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'oswac001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'peraj003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'peted003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'pillk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'reedj005', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'reids001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'smitd008', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'smitd009', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stocr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'strom001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tovaw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tropn001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'villj001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'walkt004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willm007', 'batter_hand'] = 'L'
    events_df.loc[events_df['pitcherID'] == 'greec004', 'pitcher_hand'] = 'R'
### PHI
    events_df.loc[events_df['batterID'] == 'alvaj004', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'andec001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bedrc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bohma001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bonij002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brada001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brogc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'coons001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crouh001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'efliz001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'faltb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gregd001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'haled001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hammj003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'harpb003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hasea001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'herro001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hoskr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'howas003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'jankt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'joycm001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'kenni001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kings001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kintb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'knapa001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'llovm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'marcr001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'maton001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'mccua001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'media001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'millb002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'monim001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'moorm003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'nerih001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'nolaa001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'pauld002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'realj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'romej002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'rossr003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'sancc002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'seguj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'suarr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'torrr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'velav001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'vierm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wheez001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'willl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['pitcherID'] == 'joned003', 'pitcher_hand'] = 'L'
### PIT
    events_df.loc[events_df['batterID'] == 'alfoa002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'andet002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'andet003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'braus002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'brubj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cahit001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'castr006', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'contr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'craiw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crick001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'croww001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cruzo001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'davia003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dejoc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'difow001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'evanp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fowld002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'fraza001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'frazt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hayek001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'holmc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'howas002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kellk003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kellm003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kranm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kuhlc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mearn001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mills001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'morac001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'newmk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'nogoj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'olivj003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'overc001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'oviel001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'perem005', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'peted001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'polag001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'poncc002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'reynb001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'shrec001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'spits001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stalj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'stokt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'strac001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tuckc001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'unded001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yajum001', 'batter_hand'] = 'R'
### SDN
    events_df.loc[events_df['batterID'] == 'adama002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'andrj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'camad001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'campl002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'carav001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'crisn001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cronj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'darvy001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'diazm004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hillt002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hillt002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hosme001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'hudsd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'johnp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kelak001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kim-h002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kivlp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'knehr001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'machm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'marct001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'melam001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'morea001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'musgj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'myerw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'nolaa002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'norta001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'norwj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ograb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'paddc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'pagae001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'phamt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'pomed001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ramin003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'rivaw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'snelb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'stamc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tatif002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'thomm005', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'weatr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willt003', 'batter_hand'] = 'B'
    events_df.loc[events_df['pitcherID'] == 'carpm002', 'pitcher_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'casti001', 'batter_hand'] = 'B'
    ### SFN
    events_df.loc[events_df['batterID'] == 'alvaj003', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'barac001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bartj003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'brebj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'casac001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'chatt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'crawb001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'cuetj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'davij008', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'desca001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dicka001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'dubom001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'duggs001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'estrt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'florw001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garcj006', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'gausk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'jackj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'kazms001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'lastt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'leond003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'littz001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'longe001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'longs002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'mcgej001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'menec001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'poseb001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'quinj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'roget002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ruf-d001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'sanca006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'slata001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tromc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'voslj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'wadel001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'webbl001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wislm001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wooda002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'yastm001', 'batter_hand'] = 'L'
    ## SLN
    events_df.loc[events_df['batterID'] == 'arenn001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'badeh001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'cabrg001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'carld002', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'deana001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'dejop001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'edmat001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'fernj006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'flahj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gallg001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'gantj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garcl005', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'happj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'helsr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hudsd002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'hurss001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'kim-k001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'kniza001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'leblw001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'lestj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'mcfat001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'mikom001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'milla002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'millj006', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'moliy001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'morom001', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'nootl001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'oneit001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'oviej001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'poncd001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'reyea004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'ronda001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'rondj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'sanca009', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'tapid001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'thoml002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'waina001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'webbt001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'webbt001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'whitk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'willj006', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'woodj003', 'batter_hand'] = 'R'
### WAS
    events_df.loc[events_df['batterID'] == 'adonj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'arteh001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'avila001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'avill001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'balda001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'barrt004', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'bellj005', 'batter_hand'] = 'B'
    events_df.loc[events_df['batterID'] == 'casts001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'corbp001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'escoa003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'espip001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'fedde001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'finnk001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'garcl006', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'guerj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'harpr001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'harrw002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'herny002', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'kiebc001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'lobsk001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'lucrj001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'macha003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mcgok002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'mercj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'murpp001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'nolis001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'parrg001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'pereh001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'raint003', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'roblv001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'rodrj005', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'rossj002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'sanca007', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'sotoj001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'steva001', 'batter_hand'] = 'L'
    events_df.loc[events_df['batterID'] == 'stras001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'suerw002', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'votha001', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'wilsc005', 'batter_hand'] = 'R'
    events_df.loc[events_df['batterID'] == 'zimmr001', 'batter_hand'] = 'R'

    return events_df
print('ok')

ok


In [113]:
s = time.time()
game_info_all_df = pandas.DataFrame(data=None, columns=info_columns).set_index('gameID')
game_data_all_df = pandas.DataFrame(data=None, columns=['gameID'])#.set_index('gameID')
er_all_df = pandas.DataFrame(data=None, columns=['gameID', 'pitcher', 'er'])

events_all_df = pandas.DataFrame(data=None, columns=event_columns).set_index('eventID')

players_all_df = pandas.DataFrame(data=None, columns=player_columns).set_index(['playerID'])
watchflag = False

comments_at_events = []
pinch_runners_at_events = []
badj_at_events = []
padj_at_events = []
ladj_at_events = []

all_lines_list = []

for i in range(firstyear, finalyear+1):
    
    game_info_thisyear_df = pandas.DataFrame(data=None, columns=game_info_all_df.columns)
#    game_data_thisyear_df = pandas.DataFrame(data=None, columns=game_data_all_df.columns)
    er_thisyear_df = pandas.DataFrame(data=None, columns=er_all_df.columns)
    players_thisyear_df = pandas.DataFrame(data=None, columns=players_all_df.columns)
    events_thisyear_df = pandas.DataFrame(data=None, columns=events_all_df.columns)
    
    thisyear_files = sorted(['{0:}{1:.0f}seve/'.format(eventdir, np.floor(i/10)*10)+x for x in os.listdir('{0:}{1:.0f}seve/'.format(eventdir, np.floor(i/10)*10)) if ((str(i) in x) & (x[-4:] == '.EV{0:}'.format(league)))])
    print('Found {0:,.0f} event files...'.format(len(thisyear_files)))
    for thisfile in thisyear_files[just_this_team_order:just_this_team_order+1]:
        game_info_df = pandas.DataFrame(data=None, columns=game_info_thisyear_df.columns)
#        game_data_df = pandas.DataFrame(data=None, columns=game_data_thisyear_df.columns)
        er_df = pandas.DataFrame(data=None, columns=er_thisyear_df.columns)
        events_df = pandas.DataFrame(data=None, columns=events_thisyear_df.columns)
        players_df = pandas.DataFrame(data=None, columns=players_thisyear_df.columns)

        all_lines_list = []
        print('Reading {0:}...'.format(thisfile.split('/')[-1]))
        with io.open(thisfile, 'r', encoding='utf-8') as f:
            orig_lines_list = f.readlines()
        for this_orig_line in orig_lines_list:
            all_lines_list += [x for x in this_orig_line.split('\n')]
        all_lines_list = [x for x in all_lines_list 
                          if ((x != '') and (x != '\x1a'))
                         ]

        #print('\tRead {0:,.0f} lines...'.format(len(all_lines_list)))
        
        event_dict = {}
        gamecount = 0
        eventcount = 0
        the_event_in_game = 0
        batting_team = 0
        comment_text = ''
        game_info_comment_text = ''

        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
        staged_play = False
        
        for i in range(0, len(all_lines_list)):

            if (np.mod(i,1000) == 0):
                print('\tParsing line {0:,.0f} of {1:,.0f}...'.format(i, len(all_lines_list)))
            #    print(i, all_lines_list[i])
            record_type = all_lines_list[i].split(',')[0]
            #print(record_type)
            if (record_type == 'id'):                
                if (all_lines_list[i].split(',')[1] == 'BLF191506250'):
                    #print('\n')
                    watchflag = False
                if (all_lines_list[i].split(',')[1] == 'BLF191506261'):
                    watchflag = False
                if (gamecount >= 1):   #WRITE LAST PLAY OF PREVIOUS GAME!
                    if (comment_text != ''):
                        event_dict['comments'] = comment_text
                    else:
                        event_dict['comments'] = np.nan
                    event_dict['pinch_runner_playerID'] = pinch_runner
                    event_dict['pinch_runner_replaced_playerID'] = pinch_runner_replaced
                    if (pinch_runner_flag == True):
                        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
                    events_df.loc[eventcount] = event_dict
                    eventcount += 1
                    the_event_in_game == 1
                    comment_text = ''

                    if (game_info_comment_text != ''):
                        this_game_info['comments'] = game_info_comment_text
                        game_info_comment_text = ''
                    else:
                        this_game_info['comments'] = np.nan
                    game_info_df.loc[thegameID] = this_game_info

                gamecount += 1        
                thegameID = all_lines_list[i].split(',')[1]
                if (np.mod(gamecount, 50) == 0):
                    print('\t\tReading {0:} (starts on line {1:.0f})...'.format(thegameID, i))

                players_df = players_df[players_df['visitor_or_home'].apply(lambda x: True == False)]

        #         if (thegameID == 'ANA202008100'):
        #             break
                eventcount += 1
                this_game_info = {}
                the_event_in_game = 1
                staged_play = False
#                 if (np.mod(gamecount, 50) == 1):
#                     print('\t\tParsing game {0:.0f}: {1:} (i = {2:,.0f}; eventcount = {3:,.0f})...'.format(gamecount, thegameID, i, eventcount))
        #            print('Starting Top of the 1st...')
            elif (record_type == 'version'):
                this_game_info['version'] = all_lines_list[i].split(',')[1]
            elif (record_type == 'info'):
                this_game_info[all_lines_list[i].split(',')[1]] = all_lines_list[i].split(',')[2]
            elif (record_type == 'start'):
                this_player_df = pandas.DataFrame(data=[[all_lines_list[i].split(',')[1], int(all_lines_list[i].split(',')[3]), int(all_lines_list[i].split(',')[4]), int(all_lines_list[i].split(',')[5])]], columns=player_columns).set_index('playerID')
                players_df = pandas.concat((players_df, this_player_df), axis=0)
            elif (record_type == 'data'):
                try:
                    this_er_row_df = pandas.DataFrame(data=[[thegameID, all_lines_list[i].split(',')[2], int(all_lines_list[i].split(',')[3])]], columns=er_df.columns)
                except ValueError:
                    this_er_row_df = pandas.DataFrame(data=[[thegameID, all_lines_list[i].split(',')[2], int(all_lines_list[i].split(',')[3][0:2])]], columns=er_df.columns) # Catching one line (BAL2021 line 6015) with a stray "1    a"
                er_df = pandas.concat((er_df, this_er_row_df))
            elif (record_type == 'play'):
                if ((watchflag) & (all_lines_list[i].split(',')[-1] == 'NP')):
                    print('{0:0d}: {1:}: {2:}'.format(i, all_lines_list[i], all_lines_list[i].split(',')[-1]))
                if (staged_play):
                    if (comment_text != ''):
                        event_dict['comments'] = comment_text
                    else:
                        event_dict['comments'] = np.nan
                    event_dict['pinch_runner_playerID'] = pinch_runner
                    event_dict['pinch_runner_replaced_playerID'] = pinch_runner_replaced
                    if (pinch_runner_flag == True):
                        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
                    events_df.loc[eventcount] = event_dict
                    eventcount += 1
                    the_event_in_game == 1
                    comment_text = ''
                event_dict['event_in_game'] = the_event_in_game
                event_dict['gameID'] = thegameID
                event_dict['inning'] = int(all_lines_list[i].split(',')[1])
                event_dict['visitor_or_home'] = int(all_lines_list[i].split(',')[2])
                event_dict['batterID'] = all_lines_list[i].split(',')[3]
                event_dict['pitch_count'] = all_lines_list[i].split(',')[4]
                event_dict['pitch_sequence'] = all_lines_list[i].split(',')[5]
                event_dict['theplay'] = all_lines_list[i].split(',')[6]        
                
                #print(players_df)
                
                event_dict['pitcherID'] = players_df[
                    (players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) 
                    & (players_df['fielding_position'] == 1)
                ].index.values[0]
                #print('\t{0:}'.format(event_dict['pitcherID']))
                if (save_fielders):
                    event_dict['field_C_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 2)].index.values[0]
                    event_dict['field_1B_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 3)].index.values[0]
                    event_dict['field_2B_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 4)].index.values[0]
                    event_dict['field_3B_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 5)].index.values[0]
                    event_dict['field_SS_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 6)].index.values[0]
                    event_dict['field_LF_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 7)].index.values[0]
                    event_dict['field_CF_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 8)].index.values[0]
                    event_dict['field_RF_playerID'] = players_df[(players_df['visitor_or_home'] == 1 - event_dict['visitor_or_home']) & (players_df['fielding_position'] == 9)].index.values[0]
                                        
#                     if (this_game_info['usedh'] == 'true'):  # lol it's a string, not a boolean
#                         event_dict['bat_DH_playerID'] = players_df[(players_df['visitor_or_home'] == event_dict['visitor_or_home']) & (players_df['fielding_position'] == 10)].index.values[0]
                if (batting_team != event_dict['visitor_or_home']):
        #             if (event_dict['visitor_or_home'] == 0):
        #                 inningside = 'Top'
        #             else:
        #                 inningside = 'Bottom'
        #             print('Starting {0:} of the {1:} - {2:} pitching...'.format(inningside, ordinal(event_dict['inning']), event_dict['pitcherID']))
                    batting_team = 1 - batting_team
                staged_play = True
                the_event_in_game += 1
            elif (record_type == 'com'):
                if (the_event_in_game <= 1):
                    game_info_comment_text += all_lines_list[i][4:]
                if (eventcount not in comments_at_events):
                    comments_at_events.append(eventcount)
                if (staged_play):
                    if (comment_text != ''):
                        comment_text += ' '
                    comment_text += all_lines_list[i][4:]#.replace('"', '').replace('$', '').replace('.', '')
            elif (record_type == 'sub'):
                if (watchflag):
                    print('\t{0:0d}: {1:}'.format(i, all_lines_list[i]))
                #print(players_df)
                #print('\t\t{0:}'.format(all_lines_list[i]))
                players_df = players_df.reset_index()
                sub_playerID = all_lines_list[i].split(',')[1]
                sub_team = int(all_lines_list[i].split(',')[3])
                sub_batting_order = int(all_lines_list[i].split(',')[4])
                sub_fielding_position = int(all_lines_list[i].split(',')[5])
                if (sub_team == batting_team):
                    if (sub_fielding_position == 11):
                        pass
                        #event_dict['batterID'] = sub_playerID
                    elif (sub_fielding_position == 12):
                        pinch_runners_at_events.append(eventcount)
                        [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [True, sub_playerID, players_df[(players_df['visitor_or_home'] == batting_team) & (players_df['batting_order'] == sub_batting_order)]['playerID'].values[0]]
                    else:
                        players_df.loc[(players_df['visitor_or_home'] == batting_team) & (players_df['batting_order'] == sub_batting_order), 'playerID'] = sub_playerID
                else:
                    players_df.loc[
                        (players_df['visitor_or_home'] == 1-batting_team) 
                        & (players_df['fielding_position'] == sub_fielding_position), 'playerID'] = sub_playerID
                players_df = players_df.set_index('playerID')
            elif (record_type == 'radj'):
                pinch_runners_at_events.append(eventcount)
        #         print('-------------------- RADJ ---------------------------')
        #         print(all_lines_list[i])
                [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [True, all_lines_list[i].split(',')[1], 'radj']
        #         print(pinch_runner_flag, pinch_runner, pinch_runner_replaced)
        #         print('-------------------- /RADJ ---------------------------')
            elif (record_type == 'badj'):

                if (the_event_in_game == 1):
                    #print('\tbadj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                    badj_at_events.append(eventcount)
                else:
                    #print('\tbadj at eventcount {0:,.0f}: {1:}'.format(eventcount+1, all_lines_list[i]))
                    badj_at_events.append(eventcount+1)
            elif (record_type == 'padj'):
                if (the_event_in_game == 1):
                    #print('\tpadj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                    padj_at_events.append(eventcount)
                else:
                    #print('\tpadj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                    padj_at_events.append(eventcount+1)
            elif (record_type == 'ladj'):
                #print('\tladj at eventcount {0:,.0f}: {1:}'.format(eventcount, all_lines_list[i]))
                #print(all_lines_list[i])
                ladj_at_events.append(eventcount)
            else:
                print('--------------')
                print(all_lines_list[i-1])     
                print(all_lines_list[i])     
                print(all_lines_list[i+1])     
                print('--------------')
                print('--------------')

        # record last play of final game
        if (comment_text != ''):
            event_dict['comments'] = comment_text
        else:
            event_dict['comments'] = np.nan
        event_dict['pinch_runner_playerID'] = pinch_runner
        event_dict['pinch_runner_replaced_playerID'] = pinch_runner_replaced
        if (pinch_runner_flag == True):
            [pinch_runner_flag, pinch_runner, pinch_runner_replaced] = [False, np.nan, np.nan]
        events_df.loc[eventcount] = event_dict
        eventcount += 1
        the_event_in_game == 1
        comment_text = ''        

        # record final game info and data
        if (game_info_comment_text != ''):
            this_game_info['comments'] = game_info_comment_text
        else:
            this_game_info['comments'] = np.nan
        game_info_df.loc[thegameID] = this_game_info
        
        events_df.loc[badj_at_events, 'badj'] = True        
        events_df.loc[padj_at_events, 'padj'] = True
        
        game_info_thisyear_df = pandas.concat((game_info_thisyear_df, game_info_df), axis=0)
#        game_data_thisyear_df = pandas.concat((game_data_thisyear_df, game_data_df), axis=0)
        er_thisyear_df = pandas.concat((er_thisyear_df, er_df), axis=0)
        players_thisyear_df = pandas.concat((players_thisyear_df, players_df), axis=0)
        
        events_thisyear_df = pandas.concat((events_thisyear_df, events_df), axis=0)

        print('\n')
        
    game_info_all_df = pandas.concat((game_info_all_df, game_info_thisyear_df), axis=0)
#    game_data_all_df = pandas.concat((game_data_all_df, game_data_thisyear_df), axis=0)
    er_all_df = pandas.concat((er_all_df, er_thisyear_df), axis=0)
    players_all_df = pandas.concat((players_all_df, players_thisyear_df), axis=0)

    events_all_df = pandas.concat((events_all_df, events_thisyear_df), axis=0)

    
#print('Killing in-progress events and replacing with the full file...')

game_info_df = game_info_df[game_info_df['number'].apply(lambda x: True == False)]
er_df = er_df[er_df['er'].apply(lambda x: True == False)]
players_df = players_df[players_df['visitor_or_home'].apply(lambda x: True == False)]
events_df = events_df[events_df['event_in_game'].apply(lambda x: True == False)]

game_info_df = game_info_all_df
er_df = er_all_df
players_df = players_all_df
events_df = events_all_df

er_df = er_df.reset_index(drop=True)
events_df = events_df.reset_index(drop=True)

game_info_df.index.name = 'gameID'
er_df.index.name = 'erID'
players_df.index.name = 'playerID'
events_df.index.name = 'eventID'

# print('backing up...')
game_info_df_bk = game_info_df
er_df_bk = er_df
players_df_bk = players_df
events_df_bk = events_df

e = time.time()
g = g + (e-s)
#print('\n')
print('Parsed {0:,.0f} events in {1:,.0f} minutes {2:,.0f} seconds!'.format(len(events_df), np.floor((e-s)/60), (e-s)%60))


Found 15 event files...
Reading 2021WAS.EVN...
	Parsing line 0 of 13,338...
	Parsing line 1,000 of 13,338...
	Parsing line 2,000 of 13,338...
	Parsing line 3,000 of 13,338...
	Parsing line 4,000 of 13,338...
	Parsing line 5,000 of 13,338...
	Parsing line 6,000 of 13,338...
	Parsing line 7,000 of 13,338...
		Reading WAS202107190 (starts on line 7829)...
	Parsing line 8,000 of 13,338...
	Parsing line 9,000 of 13,338...
	Parsing line 10,000 of 13,338...
	Parsing line 11,000 of 13,338...
	Parsing line 12,000 of 13,338...
	Parsing line 13,000 of 13,338...


Parsed 7,237 events in 5 minutes 36 seconds!


In [116]:
s = time.time()

print('getting from backingup...')
game_info_df = game_info_df[game_info_df['number'].apply(lambda x: True == False)]
er_df = er_df[er_df['er'].apply(lambda x: True == False)]
players_df = players_df[players_df['visitor_or_home'].apply(lambda x: True == False)]
events_df = events_df[events_df['event_in_game'].apply(lambda x: True == False)]
game_info_df = game_info_df_bk
er_df = er_df_bk
players_df = players_df_bk
events_df = events_df_bk

print('getting roster file...')
rosters_df = pandas.read_csv(processed_data_dir+'rosters.csv'.format(firstyear, finalyear), index_col='rownumber')

print('adding players not in roster file...')
rosters_df = add_special_case_roster_entries(rosters_df)


events_df = events_df.assign(theyear = pandas.to_numeric(events_df['gameID'].apply(lambda x: x[3:7]), errors='coerce', downcast='integer'))
events_df = events_df.assign(visteam = events_df.reset_index().merge(game_info_df.reset_index()[['gameID', 'visteam']], how='left', on='gameID').set_index('eventID')['visteam'])
events_df = events_df.assign(hometeam = events_df.reset_index().merge(game_info_df.reset_index()[['gameID', 'hometeam']], how='left', on='gameID').set_index('eventID')['hometeam'])


# # ### Different code for Seattle and Texas because WTF
# print('Seattle/Texas code: Scanning {0:,.0f} events...'.format(len(events_df)))
# missing_batters_list = []
# missing_pitchers_list = []
# for ix, thisrow in events_df.iterrows():
#     if (np.mod(ix, 100) == 0):
#         print('\tix = {0:.0f}...'.format(ix))
# # batters        
#     if (len(rosters_df[(rosters_df['playerID'] == thisrow['batterID']) & (rosters_df['theyear'] == 2021)]['batting_hand'].tolist()) == 0):
#         if (thisrow['batterID'] not in missing_batters_list):
#             missing_batters_list.append(thisrow['batterID'])
#     else:
#         events_df.loc[ix, 'batter_hand'] = rosters_df[(rosters_df['playerID'] == thisrow['batterID']) & (rosters_df['theyear'] == 2021)]['batting_hand'].values[0]
# # pitchers
#     if (len(rosters_df[(rosters_df['playerID'] == thisrow['pitcherID']) & (rosters_df['theyear'] == 2021)]['throwing_hand'].tolist()) == 0):
#         if (thisrow['pitcherID'] not in missing_pitchers_list):
#             missing_pitchers_list.append(thisrow['pitcherID'])
#     else:
#         events_df.loc[ix, 'pitcher_hand'] = rosters_df[(rosters_df['playerID'] == thisrow['pitcherID']) & (rosters_df['theyear'] == 2021)]['throwing_hand'].values[0]
# # print('doon')
# # pprint(sorted(missing_batters_list))
# # print('\n')
# # pprint(sorted(missing_pitchers_list))

## Everyone except Seattle and Texas
events_df.loc[(events_df['visitor_or_home'] == 0), 'batter_hand'] = events_df[(events_df['visitor_or_home'] == 0)].reset_index()[['eventID', 'batterID', 'theyear', 'visteam']].merge(rosters_df.reset_index()[['playerID', 'theyear', 'team', 'batting_hand']], how='left', left_on=['batterID', 'theyear', 'visteam'], right_on=['playerID', 'theyear', 'team']).set_index('eventID')['batting_hand']
events_df.loc[(events_df['visitor_or_home'] == 1), 'batter_hand'] = events_df[(events_df['visitor_or_home'] == 0)].reset_index()[['eventID', 'batterID', 'theyear', 'visteam']].merge(rosters_df.reset_index()[['playerID', 'theyear', 'team', 'batting_hand']], how='left', left_on=['batterID', 'theyear', 'visteam'], right_on=['playerID', 'theyear', 'team']).set_index('eventID')['batting_hand']

print('finding pitcher handedness...')
events_df.loc[(events_df['visitor_or_home'] == 1), 'pitcher_hand'] = events_df[(events_df['visitor_or_home'] == 1)].reset_index()[['eventID', 'pitcherID', 'theyear', 'visteam']].merge(rosters_df.reset_index()[['playerID', 'theyear', 'team', 'throwing_hand']], how='left', left_on=['pitcherID', 'theyear', 'visteam'], right_on=['playerID', 'theyear', 'team']).set_index('eventID')['throwing_hand']
events_df.loc[(events_df['visitor_or_home'] == 0), 'pitcher_hand'] = events_df[(events_df['visitor_or_home'] == 0)].reset_index()[['eventID', 'pitcherID', 'theyear', 'hometeam']].merge(rosters_df.reset_index()[['playerID', 'theyear', 'team', 'throwing_hand']], how='left', left_on=['pitcherID', 'theyear', 'hometeam'], right_on=['playerID', 'theyear', 'team']).set_index('eventID')['throwing_hand']

print('fixing identified special cases...')
events_df = fix_special_case_hands(events_df)

#print('accounting for switch hitters...')
events_df.loc[(events_df['batter_hand'] == 'B') & (events_df['pitcher_hand'] == 'R'), 'batter_hand'] = 'L'
events_df.loc[(events_df['batter_hand'] == 'B') & (events_df['pitcher_hand'] == 'L'), 'batter_hand'] = 'R'

#print('app:)lying adjustments...')
#print('\n')
pitcher_hand_switcher_df = pandas.DataFrame(data=events_df[(events_df['padj'] == True)]['pitcher_hand'], columns=['pitcher_hand'])
pitcher_hand_switcher_df = pitcher_hand_switcher_df.assign(pitcher_hand_switched = np.nan)
pitcher_hand_switcher_df.loc[pitcher_hand_switcher_df['pitcher_hand'] == 'L', 'pitcher_hand_switched'] = 'R'
pitcher_hand_switcher_df.loc[pitcher_hand_switcher_df['pitcher_hand'] == 'R', 'pitcher_hand_switched'] = 'L'
events_df.loc[(events_df['padj'] == True), 'pitcher_hand'] = pitcher_hand_switcher_df['pitcher_hand_switched']

batter_hand_switcher_df = pandas.DataFrame(data=events_df[(events_df['badj'] == True)]['batter_hand'], columns=['batter_hand'])
batter_hand_switcher_df = batter_hand_switcher_df.assign(batter_hand_switched = np.nan)
batter_hand_switcher_df.loc[batter_hand_switcher_df['batter_hand'] == 'L', 'batter_hand_switched'] = 'R'
batter_hand_switcher_df.loc[batter_hand_switcher_df['batter_hand'] == 'R', 'batter_hand_switched'] = 'L'
events_df.loc[(events_df['badj'] == True), 'batter_hand'] = batter_hand_switcher_df['batter_hand_switched']

print('\n')
print('No batter hand found for {0:,.0f} batters...'.format(len(events_df[events_df['batter_hand'].isnull()]['batterID'].drop_duplicates())))

missing_hand_batters_list = events_df[events_df['batter_hand'].isnull()]['batterID'].drop_duplicates().sort_values().tolist()
missing_hand_batters_list = sorted(missing_hand_batters_list)
missing_hand_batters_list

for ix, thisrow in rosters_df[
    (rosters_df['playerID'].isin(missing_hand_batters_list)) 
    ][['playerID', 'batting_hand']].drop_duplicates().sort_values(by='playerID').iterrows():
    print("    events_df.loc[events_df['batterID'] == '{0:}', 'batter_hand'] = '{1:}'".format(thisrow['playerID'], thisrow['batting_hand']))

for ix, thisbatterID in events_df[
    events_df['batter_hand'].isnull()
]['batterID'].sort_values().drop_duplicates().iteritems():
    print(thisbatterID)

print('\n')
print('No pitcher hand found for {0:,.0f} pitchers...'.format(len(events_df[events_df['pitcher_hand'].isnull()]['pitcherID'].drop_duplicates())))

missing_hand_pitchers_list = events_df[(events_df['pitcher_hand'].isnull())]['pitcherID'].drop_duplicates().sort_values().tolist()
# # #missing_hand_pitchers_list

for ix, thisrow in rosters_df[
    (rosters_df['playerID'].isin(missing_hand_pitchers_list)) 
    ][['playerID', 'throwing_hand']].drop_duplicates().sort_values(by='playerID').iterrows():
    print("    events_df.loc[events_df['pitcherID'] == '{0:}', 'pitcher_hand'] = '{1:}'".format(thisrow['playerID'], thisrow['throwing_hand']))

for ix, thispitcherID in events_df[
    events_df['pitcher_hand'].isnull()
]['pitcherID'].sort_values().drop_duplicates().iteritems():
    print(thispitcherID)

e = time.time()
g = g + (e-s)
print('Done in {0:,.0f} minutes {1:,.0f} seconds!'.format(np.floor((e-s)/60), (e-s)%60))


getting from backingup...
getting roster file...
adding players not in roster file...
finding pitcher handedness...
fixing identified special cases...


No batter hand found for 0 batters...


No pitcher hand found for 0 pitchers...
Done in 0 minutes 2 seconds!


In [ ]:

# rosters_df[rosters_df['playerID'].isin(
#     ['bowdb001', 'lawrj002', 'rodrc002']
# )].sort_values(by=['playerID', 'theyear'])


# print(events_df[events_df['batterID'] == 'santa003'].groupby('visteam').size())
# print('\n')
# print(events_df[events_df['batterID'] == 'santa003'].groupby('hometeam').size())

# print(events_df[events_df['pitcherID'] == 'leond003'].groupby('visteam').size())
# print('\n')
# print(events_df[events_df['pitcherID'] == 'leond003'].groupby('hometeam').size())

# rosters_df[
#     (rosters_df['theyear'] == 2021)
#     & (rosters_df['batting_hand'] == 'B')
#     & (rosters_df['playerID'].isin(

#     ))
# ].sort_values(by='playerID')['playerID'].drop_duplicates().tolist()



#events_df[events_df['batterID'] == 'shorz001'][['batterID', 'visitor_or_home', 'visteam', 'hometeam']]

#events_df.columns
#events_df[['batterID', 'batter_hand']].drop_duplicates().groupby('batterID').size().sort_values(ascending=False)


In [117]:
s = time.time()
print('saving data...')
# if (len(just_this_team) > 0):
#     game_info_df.to_csv('game_info_{0:}_{1:}_{2:}.csv'.format(firstyear, finalyear, just_this_team), encoding='utf-8')
#     er_df.to_csv('er_{0:}_{1:}_{2:}.csv'.format(firstyear, finalyear, just_this_team), encoding='utf-8')
#     events_df.to_csv('events_{0:}_{1:}_{2:}.csv'.format(firstyear, finalyear, just_this_team), encoding='utf-8')
# #     events_df[events_df.index.isin(badj_at_events)].to_csv('badj_events_{0:}_{1:}_{2:}.csv'.format(firstyear, finalyear, just_this_team), encoding='utf-8')
# #     events_df[events_df.index.isin(padj_at_events)].to_csv('padj_events_{0:}_{1:}_{2:}.csv'.format(firstyear, finalyear, just_this_team), encoding='utf-8')
# #     events_df[events_df.index.isin(ladj_at_events)].to_csv('ladj_events_{0:}_{1:}_{2:}.csv'.format(firstyear, finalyear, just_this_team), encoding='utf-8')
# else:

game_info_df.to_csv('game_info_{0:}_{1:}{2:}_{3:}.csv'.format(firstyear, finalyear, league.lower(), just_this_team.lower()), encoding='utf-8')
er_df.to_csv('er_{0:}_{1:}{2:}_{3:}.csv'.format(firstyear, finalyear, league.lower(), just_this_team.lower()), encoding='utf-8')
events_df.to_csv('events_{0:}_{1:}{2:}_{3:}.csv'.format(firstyear, finalyear, league.lower(), just_this_team.lower()), encoding='utf-8')
# if (len(badj_at_events) > 0):
#     events_df[events_df.index.isin(badj_at_events)].to_csv('badj_events_{0:}_{1:}{2:}_{3:}.csv'.format(firstyear, finalyear, league.lower(), just_this_team.lower()), encoding='utf-8')
# else:
#     print('no badj found')
# if (len(padj_at_events) > 0):
#     events_df[events_df.index.isin(padj_at_events)].to_csv('padj_events_{0:}_{1:}{2:}_{3:}.csv'.format(firstyear, finalyear, league.lower(), just_this_team.lower()), encoding='utf-8')
# else:
#     print('no padj found')
# if (len(ladj_at_events) > 0):
#     events_df[events_df.index.isin(ladj_at_events)].to_csv('ladj_events_{0:}_{1:}{2:}_{3:}.csv'.format(firstyear, finalyear, league.lower(), just_this_team.lower()), encoding='utf-8')
# else:
#     print('no ladj found')

e = time.time()
g = g + (e-s)
print('Done in {0:,.0f} minutes {1:,.0f} seconds!'.format(np.floor(g/60), g%60))


saving data...
Done in 5 minutes 41 seconds!


In [ ]:
# AL
# ANA 0
# BAL 1
# BOS 2
# CHA 3
# CLE 4
# DET 5
# HOU 6
# KCA 7
# MIN 8
# NYA 9
# OAK 10
# SEA 11
# TBA 12
# TEX 13
# TOR 14

# NL
# ARI 0
# ATL 1
# CHN 2
# CIN 3
# COL 4
# LAN 5
# MIA 6
# MIL 7
# NYN 8
# PHI 9
# PIT 10
# SDN 11
# SFN 12
# SLN 13
# WAS 14






In [ ]:
# ### 1915
#     events_df.loc[events_df['batterID'] == 'bradh101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'browm102', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'conls101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'coopc101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'crand101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'eaklc101', 'batter_hand'] = '?'
#     events_df.loc[events_df['batterID'] == 'englc103', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'hartg102', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'mannl101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'simom102', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'wistt101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'blaiw101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'chadc101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'conls101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'esmoj101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'flacm101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'gingj101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'kaufb101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'kerrd102', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'knete101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'lordh101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'maina101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'mckeb101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'quinj102', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'browm102', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'crand101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'conls101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'flacm101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'gingj101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'knete101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'maina101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'quinj102', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'gagne101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'whitc102', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'woodd101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'gagne101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'whitc102', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'woodd101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'rheac101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'delaf101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'reedt101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'delaf101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'mannl101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'bradh101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'simom102', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'blaiw101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'mckeb101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'rheac101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hartg102', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'millh102', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'harso101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'neffd101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'parkp101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'walsd101', 'batter_hand'] = 'B'
#     events_df.loc[events_df['pitcherID'] == 'harso101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'parkp101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'walsd101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'brieb101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'bushd102', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'cartp101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'cotte101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'dalyt102', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'daush101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'gregv101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'hamie101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'jackj101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'koobe101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'lappj101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'lewid101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'lewid101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'mortg101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'mullc101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'russr102', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'speat101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'younr103', 'batter_hand'] = 'B'
#     events_df.loc[events_df['pitcherID'] == 'cartp101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'cotte101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'dalyt102', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'daush101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'engej101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'gregv101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'hamie101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'koobe101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'mortg101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'russr102', 'pitcher_hand'] = 'L'    
#     events_df.loc[events_df['batterID'] == 'bresr101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'engej101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'lowdg101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'bresr101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'lappj101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'lowdg101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'mullc101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'younr103', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'eschj101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'eschj101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'bradn101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'bradn101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'hendt101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'brieb101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'comsr101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'crans102', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'richj103', 'batter_hand'] = 'B'
#     events_df.loc[events_df['pitcherID'] == 'comsr101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'richj103', 'pitcher_hand'] = 'R'    
#     events_df.loc[events_df['batterID'] == 'ecclh101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'ecclh101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'barbt101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'jamic101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'beckb101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'braif101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'chenl101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'demaa101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'dougp101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'egand102', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'fleta101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'hargb101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'jameb102', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'johnd107', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'lobeh101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'mccal102', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'robed103', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'schuj101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'snydf101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'beckb101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'braif101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'chenl101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'demaa101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'dougp101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'jameb102', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'mccal102', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'schuj101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'dolac102', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'mclap101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'shanj102', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'stanp101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'wagnj101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'dolac102', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'mclap101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'stanp101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'collz101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'stanp101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'leark101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'shanj102', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'oescj101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'hargb101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'oescj101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'corrr101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'corrr101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'leark101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'callr101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'callr101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'wagnj101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'robed103', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'schef104', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'egand102', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'clarf101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'daubh101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'boarc101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'lamlf101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'lamlf101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'boarc101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'schef104', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'englc103', 'pitcher_hand'] = 'R'
    
#     #1916
#     events_df.loc[events_df['batterID'] == 'chapr101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'culln101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'fostr103', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'fullf101', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'gallb103', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'henro101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'highh101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'judgj101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'kingl101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'lavad101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'maysc101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'nunal101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'seveh101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'shanh101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'walkt101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'weavb101', 'batter_hand'] = 'B'
#     events_df.loc[events_df['pitcherID'] == 'culln101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'danfd101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'fostr103', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'gallb103', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'maysc101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'boonl101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'danfd101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'grimc101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'hasbz101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'morig101', 'batter_hand'] = '?'
#     events_df.loc[events_df['batterID'] == 'morig101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'groob101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'berga101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'klepe101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'klepe101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'daltj101', 'batter_hand'] = 'R'    
#     events_df.loc[events_df['batterID'] == 'groob101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'morrb102', 'batter_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'morrb102', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'clemv101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'bendc101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'bohns101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'coopw101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'cravg101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'cutsg101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'dooir101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'everj102', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'fiscw101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'getzg101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'herzb101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'holkw101', 'batter_hand'] = 'B'
#     events_df.loc[events_df['batterID'] == 'jasph101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'killb101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'knabo101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'marar101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'merkf101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'ruckn101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'schua102', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'tylel101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'warmc101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'warnh102', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'zimmh101', 'batter_hand'] = 'R'    
#     events_df.loc[events_df['pitcherID'] == 'bendc101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'coopw101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'jasph101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'ruckn101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'schua102', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'tylel101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'warmc101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'currm101', 'batter_hand'] = 'R'    
#     events_df.loc[events_df['pitcherID'] == 'currm101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'chapl101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'clemc101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'gleab101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'daleg101', 'batter_hand'] = 'R'
#     events_df.loc[events_df['batterID'] == 'pearg101', 'batter_hand'] = 'L'
#     events_df.loc[events_df['pitcherID'] == 'daleg101', 'pitcher_hand'] = 'R'
#     events_df.loc[events_df['pitcherID'] == 'pearg101', 'pitcher_hand'] = 'L'
#     events_df.loc[events_df['batterID'] == 'batsb101', 'batter_hand'] = 'L'

